In [1]:
include("../src/JuliaMBD.jl")

Main.JuliaMBD

In [2]:
using DifferentialEquations
using Plots
import .JuliaMBD
import .JuliaMBD: @model, @parameter, @block, @connect, @scope

In [6]:
## MSD

@model MSD begin
    @parameter begin
        M
        D
        k
        g = 9.8
    end
    @block begin
        in1 = JuliaMBD.Inport(:in1)
        out1 = JuliaMBD.Outport(:out1)
        constant1 = JuliaMBD.Constant(value = :(M*g))
        gain1 = JuliaMBD.Gain(K = :D)
        gain2 = JuliaMBD.Gain(K = :k)
        gain3 = JuliaMBD.Gain(K = :(1/M))
        int1 = JuliaMBD.Integrator(sin = JuliaMBD.OutPort(:int1sin), sout = JuliaMBD.OutPort(:int1sout))
        int2 = JuliaMBD.Integrator(initialcondition = :(M*g/k), sin = JuliaMBD.OutPort(:int2sin), sout = JuliaMBD.OutPort(:int2sout))
        sum1 = JuliaMBD.Add(signs=[:+, :+, :-, :-])
    end
    @connect begin
        int1.out => sum1.in1
        constant1.out => sum1.in2
        gain1.out => sum1.in3
        gain2.out => sum1.in4
        sum1.out => gain3.in
        gain3.out => int1.in
        int1.out => int2.in
        int1.out => gain1.in
        int2.out => gain2.in
        int2.out => out1.in
    end
    @scope begin
        out1.in => x
        gain3.out => x2
    end
end

MSD_pfunc (generic function with 1 method)

In [7]:
M = 10
D = 2
k = 3
MSD_sfunc(3,4,3)

(8.3, 3, 0)

In [15]:
## test

@model Test begin
    @parameter begin
        M
        D
        k
        f
        p_cycle
        p_width
    end
    @block begin
        msd = MSD(M=:M, D=:D, k=:k)
        pulse = JuliaMBD.PulseGenerator(amplitude=:f, period=:p_cycle, pulsewidth=:p_width, phasedelay=10.0)
    end
    @connect begin
        pulse.out => msd.in1
    end
    @scope begin
        pulse.out => F
    end
end

Test_pfunc (generic function with 1 method)

In [19]:
b = Test(M=10, D=1, k=10, f=100, p_cycle=20, p_width=50)
JuliaMBD.expr_sfunc(b)
# Test_sfunc(1,2, M=10, D=10, k=10, f=100, p_cycle=20, p_width=50, time=10000)

:(function Test_sfunc(int1sin, int2sin; M = 10, D = 1, k = 10, f = 100, p_cycle = 20, p_width = 50, g = 9.8, time = 0)
      var"##718" = f
      var"##685" = M * g
      var"##684" = var"##685"
      begin
          var"##707" = var"##684"
      end
      begin
          var"##706" = int1sin
          var"##712" = int1sin
          var"##713" = int1sin
      end
      var"##702" = var"##706"
      var"##720" = p_width
      var"##697" = (nothing, nothing)
      var"##719" = p_cycle
      var"##691" = k
      var"##694" = 1 / M
      var"##703" = var"##707"
      begin
          var"##714" = int2sin
          var"##715" = int2sin
      end
      var"##689" = var"##714"
      var"##700" = (nothing, nothing)
      var"##683" = var"##715"
      out1 = var"##683"
      begin
      end
      var"##696" = 0.0
      var"##721" = 10.0
      var"##698" = var"##712"
      begin
          var"##722" = time
      end
      var"##688" = D
      var"##690" = var"##691" * var"##689"
      begin
     

In [11]:
# eval(JuliaMBD.expr_sfunc(b))
# eval(JuliaMBD.expr_ofunc(b))
# eval(JuliaMBD.expr_ifunc(b))
# eval(JuliaMBD.expr_pfunc(b))
m = JuliaMBD.ODEModel(
    b,
    eval(JuliaMBD.expr_odemodel_pfunc(b)),
    eval(JuliaMBD.expr_odemodel_ifunc(b)),
    eval(JuliaMBD.expr_odemodel_sfunc(b)),
    eval(JuliaMBD.expr_odemodel_ofunc(b))
);

In [12]:
@time begin
    result = JuliaMBD.simulate(m, (0.0, 60.0))
    plot(result, layout=(3,1))
end

UndefVarError: UndefVarError: f not defined